# Lesson 3: Building an Agent Reasoning Loop

## Setup

In [1]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Load the data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

## Setup the Query Tools

In [4]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("pdf/metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [5]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [6]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [7]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include the Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities in the software development process, such as generating Standardized Operating Procedures (SOPs), designing system architecture, breaking down tasks, coding, and testing. These roles collaborate to streamline workflows, ensure accurate problem-solving, and enhance code generation quality.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
Agents in MetaGPT communicate with each other through structured communication interfaces, a shared message pool, and a publish-subscribe mecha

In [8]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: pdf/metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2025-02-09
last_modified_date: 2025-02-09

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4, Jinlin Wang1, Zili Wang, Steven Ka Shing Yau5, Zijuan Lin4,
Liyang Zhou6, Chenyu Ran1, Lingfeng Xiao1,7, Chenglin Wu1†, J¨urgen Schmidhuber2,8
1DeepWisdom, 2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University, 4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University, 6University of Pennsylvania,
7University of California, Berkeley, 8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex t

In [9]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and a self-generated SoftwareDev dataset. HumanEval consists of 164 handwritten programming tasks, while MBPP comprises 427 Python tasks. The SoftwareDev dataset contains 70 representative examples of software development tasks covering diverse scopes such as mini-games, image processing algorithms, and data visualization.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and a self-generated SoftwareDev dataset. HumanEval consists of 164 handwritten programming tasks, MBPP comprises 427 Python tasks, and the SoftwareDev dataset contains 70 representative examples of software development tasks.


In [10]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results over HumanEval dataset", "page_numbers": ["7"]}
=== Function Output ===
MetaGPT achieved a pass rate of 85.9% and 87.7% over the HumanEval dataset.
=== LLM Response ===
MetaGPT achieved a pass rate of 85.9% and 87.7% over the HumanEval dataset.


## Lower-Level: Debuggability and Control

In [11]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [12]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [13]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include the Product Manager, Architect, Project Manager, Engineers, and QA Engineer. Each role has specific responsibilities in the software development process, such as generating product requirements, designing system architecture, breaking down tasks, developing code, and ensuring software quality through unit testing.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
Agents in MetaGPT communicate with each other through structured communication interfaces, shared message pools, and a publish-subscribe mechanism. This structured approach ensures that agents provide necessary outputs based on their

In [14]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task f00e82f3-569c-4a9a-ad16-b9d0cd1cf94a: 1
The agent roles in MetaGPT include the Product Manager, Architect, Project Manager, Engineers, and QA Engineer. Each role has specific responsibilities in the software development process, such as generating product requirements, designing system architecture, breaking down tasks, developing code, and ensuring software quality through unit testing.


In [15]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task f00e82f3-569c-4a9a-ad16-b9d0cd1cf94a: 1


TaskStep(task_id='f00e82f3-569c-4a9a-ad16-b9d0cd1cf94a', step_id='bae366e0-6b0e-46e8-bcd7-6499418439a4', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [16]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents share information in MetaGPT"}
=== Function Output ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool and a publish-subscribe mechanism. This system allows agents to exchange structured messages directly and subscribe to relevant information based on their roles, ensuring efficient and transparent access to necessary information within the multi-agent framework.


In [17]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool and a publish-subscribe mechanism. This system allows agents to exchange structured messages directly and subscribe to relevant information based on their roles, ensuring efficient and transparent access to necessary information within the multi-agent framework.
True


In [18]:
response = agent.finalize_response(task.task_id)

In [20]:
print(str(response))

Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool and a publish-subscribe mechanism. This system allows agents to exchange structured messages directly and subscribe to relevant information based on their roles, ensuring efficient and transparent access to necessary information within the multi-agent framework.
